In [2]:
import requests
import json
import pandas as pd
import sys
import datetime
import time
import re
import os
from dotenv import load_dotenv

import settings

from clickhouse_driver import Client
from clickhouse_key import ch_key

In [4]:
client = Client(ch_key['db_host'],
               user=ch_key['db_user'],
               password=ch_key['db_password'],
               verify=False,
               database=ch_key['db_name'], settings={'use_numpy': True})

In [5]:
pd.set_option('display.max_columns', None)

In [4]:
# последние даты / подстановка дат минус 21 день
# они же date_from

def df(result, columns):
    col = []
    for i in columns:
        col.append(i[0])
        df = pd.DataFrame(result,columns=col)
    return df    
        
def date(date):
    # минусуем от макс.даты 21 день
    date  = (datetime.datetime.strptime(date['date'][0],"%Y-%m-%d").date()-datetime.timedelta(days=21)) 
    return date

# ВООБЩЕ ХЗ НАДО ЛИ ВЫТАСКИВАТЬ МАКС.ДАТУ ИЗ БД. МБ ПРОСТО ФИКСИРОВАТЬ ДАТУ СТАРТАБ КАК СЕГОДНЯ МИНУС 21 ДЕНЬ
# УДАЛЕНИЕ ИЗ БД ПЕРЕД ЗАЛИВКОЙ: СЕГОДНЯ МИНУС 21 ДЕНЬ.

# result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
# FROM external.MT_ADS WHERE type = 'Классифайд' ''', with_column_types = True)
# data = df(result, columns)
# sec_date1 = date(data)

# result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
# FROM external.MT_ADS WHERE type = 'Новостройки' ''', with_column_types = True)
# data = df(result, columns)
# new_date1 = date(data)

# result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
# FROM external.MT_ADS WHERE type = 'Mobile' ''', with_column_types = True)
# data = df(result, columns)
# mob_date1 = date(data)

# result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
# FROM external.MT_ADS WHERE type = 'B2B' ''', with_column_types = True)
# data = df(result, columns)
# # b2b_date1 = date(data)

# result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
# FROM external.MT_ADS WHERE type = Журнал' ''', with_column_types = True)
# data = df(result, columns)
# # jor_date1 = date(data)

In [12]:
# ПЕРЕЗАПИСЬ - 21 день (согласно овакс)
sec_date1 = '2022-10-31'
new_date1 = '2022-10-31'
mob_date1 = '2022-10-31'
jor_date1 = '2022-10-31'
b2b_date1 = '2022-10-31'

In [13]:
keys = [settings.API_KEY_NEW, settings.API_KEY_SEC, settings.API_KEY_B2B, settings.API_KEY_MOB, settings.API_KEY_JOR]
values = [sec_date1, new_date1, b2b_date1, mob_date1, jor_date1]
tokens_dates = {k:v for k, v in zip(keys, values)}
tokens_dates

{'bEEpM78cJuXgrmbi2EI6wBhfT77OruwHMB1HFKewmQV4My4CiHitmcWzwcvsqfSKUt9eiSB4vLSXMytT18VZ4EQaGCrZyrZrmEIHmHsRYUVodTfZDwTr8AnKh5vFwUktD9OWXylR2J9XoMdOXU8S0DDAeCYppgtxanY88LrHfrKNebM14MjsJ4ZRIcFAYHZcx2odrILrpam1CZP3lPrqrzrS3sSeX0QgO4p34hU8Jy5aAWoE9LxVAf3ld8FP': '2022-10-31',
 'vty3a3xmwV0uovRm0FWG88HIpCa7QNCXvibtL6EMsLxUQMYX7hw4TXeLP5NBWeuCYU0rWUXohKItm8QJNnAVM2nEdyYzLBSBHcOqCoxqgvuwZd0uYTBcLfPQnbYlmLkdVzKP4Ak5rJfs6YBRw81YXNc8K9fZ2yULAGaZpffYCdDnF8C8y1qduciTaqsOsdYkFqRM2UYixjyw5vzNkMSeQ3CdkSMkVQ7': '2022-10-31',
 'F9K0cg4K83bphw3WWBcQVRTSv7r01YVupG7YbYiopl9VHaL1Zrn2SNZLZtrs9o97K40lMywmkVhSadG6qGfMduEw5ZYqDgXv1ttvFYlTBD8btzmAwMkZK20xezRghnPaIKfJCLKidj8BtW9kGC762p2lCzwq0EyeBhWdgPIYEj1uyxeLlaLqN0vJSFC6z4zEudn6YmwE8w6kOp4F': '2022-10-31',
 'en3Wq14bPGaCCQghkC5cEhwVbo5ArDya9FBBvLy6Mpw7nNzfdIGBruxAnUSwYAX8S4sZZWuIIHrmIvkKXnhXXwMrIV6NfyL530Ba3csvaMAJuS0U95AeZ6ydDer1u7bixIJd9bqmeI2s9fK4qNrerrhiwOy4VJKOboHxfOxmP5BIQvitPwJELTy46kr3yLz5Y6bbi0ZqEWk': '2022-10-31',
 '6BJOcPpWglXydLlHiVB5qg33HuTFMrEjvYaY

In [14]:
date_to = datetime.datetime.today().date() - datetime.timedelta(days=1)
date_to

datetime.date(2022, 11, 23)

In [15]:
def money(date_to, tokens_dates):
    try:
        data_you_need=pd.DataFrame()
        for i, y in tokens_dates.items():
            # это нужно для определения в дальнейшем типа продукта (Новостройки, Вторичка, Мобилка.....)
            headers = {"Authorization": "Bearer " + i}
            params = {
                "date_from": y,
                "date_to": date_to,
                "metrics": 'base'
            }
            url4 = "https://target.my.com/api/v3/user.json"
            res4 = requests.get(url4, params=params, headers=headers)
            load4 = res4.json()
            user = load4['username']

            url1 = "https://target.my.com/api/v2/statistics/banners/day.json" 
            money = requests.get(url1, params=params, headers=headers)
            load = money.json()
            time.sleep(1.5)
            in_total = []
            alls = load["items"]
            for i in alls:
                ids = i["id"]
                date = i["rows"]
                for x in date:
                    x.update({
                        "id": ids        
                    })
                    rows = pd.json_normalize(x, sep=" ").to_dict("records")
                    in_total += rows
            df = pd.DataFrame(in_total)
            df['base spent'] = df['base spent'].astype(float)
            rep_filt = df[(df['base spent'] > 0.0)]
            rep_filt = rep_filt.reset_index(drop=True)
            rep_filt['adAccount'] = user
            data_you_need=data_you_need.append(rep_filt,ignore_index=True)
    except (KeyError, AttributeError) as e:
                pass
    return data_you_need

In [16]:
money = money(date_to, tokens_dates)

In [17]:
money

,date,id,base shows,base clicks,base goals,base spent,base cpm,base cpc,base cpa,base ctr,base cr,adAccount
0,2022-10-31,111221394,654,1,0,14.32,21.9,14.32,0,0.152905,0.000000,8ee847846c@agency_client
1,2022-10-31,111221397,57,1,1,15.00,263.16,15,15,1.754386,100.000000,8ee847846c@agency_client
2,2022-10-31,111221624,299,11,5,108.94,364.35,9.9,21.79,3.678930,45.454545,8ee847846c@agency_client
3,2022-10-31,111221625,104,2,0,20.00,192.31,10,0,1.923077,0.000000,8ee847846c@agency_client
4,2022-10-31,111221626,175,4,1,40.00,228.57,10,40,2.285714,25.000000,8ee847846c@agency_client
...,...,...,...,...,...,...,...,...,...,...,...,...
192,2022-11-01,121595516,293,1,1,15.92,54.35,15.92,15.92,0.341297,100.000000,ef427e7659@agency_client
193,2022-10-31,121595517,6636,16,0,862.79,130.02,53.92,0,0.241109,0.000000,ef427e7659@agency_client
194,2022-11-01,121595517,606,0,0,35.81,59.09,0,0,0.000000,0.000000,ef427e7659@agency_client
195,2022-10-31,121595518,2251,9,1,198.64,88.25,22.07,198.64,0.399822,11.111111,ef427e7659@agency_client


In [18]:
def find_pattern(string, pattern):
    if re.search(pattern, string):
        return re.search(pattern, string).group(1)

def names(date_to, tokens_dates):
    try:
        data_you_need=pd.DataFrame()
        for i, y in tokens_dates.items():        
            headers = {"Authorization": "Bearer " + i}
            params = {
                "date_from": y,
                "date_to": date_to,
                "metrics": 'base'
            }
            url3 = "https://target.my.com/api/v2/campaigns.json?fields=id,name,utm&limit=250" # увеличила лимит с 200 до 250
            res3 = requests.get(url3, params=params, headers=headers)
            load3 = res3.json()
            tot = load3['items']
            b = pd.DataFrame(tot)
            b = b.rename(columns={"id": "campaign_id"})
            data_you_need=data_you_need.append(b,ignore_index=True)
            pettern_source = '.*utm_source=(.*?)&.*'
            pattern_medium = '.*utm_medium=(.*?)&.*'
            pattern_campaign = '.*utm_campaign=(.*?)&.*'
            pattern_content = '.*utm_content=(.*?)&.*'
            pattern_term = '.*utm_term=(.*?)$'
            data_you_need['utm'] = data_you_need['utm'].astype(str)
            data_you_need['utm_source'] = data_you_need['utm'].apply(lambda x: find_pattern(x,pettern_source)).astype(str)
            data_you_need['utm_medium'] = data_you_need['utm'].apply(lambda x: find_pattern(x,pattern_medium)).astype(str)
            data_you_need['utm_campaign'] = data_you_need['utm'].apply(lambda x: find_pattern(x,pattern_campaign)).astype(str)
            data_you_need['utm_content'] = data_you_need['utm'].apply(lambda x: find_pattern(x,pattern_content)).astype(str)
            data_you_need['utm_term'] = data_you_need['utm'].apply(lambda x: find_pattern(x,pattern_term)).astype(str)
    except (KeyError, AttributeError) as e:
                pass
    return data_you_need

names = names(date_to, tokens_dates)

In [19]:
names

,campaign_id,name,utm,utm_source,utm_medium,utm_campaign,utm_content,utm_term
0,41617387,Квиз_6_вопросов_МСК_МЖ30-49_Контекст_МОБ,None,None,None,None,None,None
1,41625259,Квиз_6_вопросов_Видео_1_МСК_М30-39_Моб_Инт:Нов...,None,None,None,None,None,None
2,42305118,Квиз_6_вопросов_СПБ_МЖ30-49_Контекст_МОБ,None,None,None,None,None,None
3,43291310,СПБ_Парколово_МЖ30+_МОБ_Мультик_Шир,None,None,None,None,None,None
4,43294245,СПБ_Парколово_МЖ30+_МОБ_Мультик_Шир-2,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
155,57999503,Дима happy Мск Android Видео,None,None,None,None,None,None
156,58000821,Дима happy 2 Мск Android Видео,None,None,None,None,None,None
157,58002574,Дима happy 3 Мск Android Видео,None,None,None,None,None,None
158,58003014,Дима happy 4 Мск Android Видео,None,None,None,None,None,None


In [21]:
def id_list(money_id):
    lst = []
    for i in money_id['id']:
        lst.append(i)
    a = lst
    return a

id_list = id_list(money)

def urls(id_list, date_to, tokens_dates):
    data_you_need=pd.DataFrame()
    for x, y in tokens_dates.items(): 
        utms_upd = []
        for i in id_list:        
            headers = {"Authorization": "Bearer " + x}            
            params = {
                "date_from": y,
                "date_to": date_to,
                "metrics": 'base'
                }
            url2 = f"https://target.my.com/api/v2/banners/{i}.json?fields=id,campaign_id,urls&limit=200"
            res2 = requests.get(url2, params=params, headers=headers)
            time.sleep(1.0)
            load2 = res2.json()
            utms_upd.append(load2)        
            a = pd.DataFrame(utms_upd)
        data_you_need=data_you_need.append(a,ignore_index=True)            
    return data_you_need

urls = urls(id_list, date_to, tokens_dates)

In [22]:
urls = urls.drop("error", axis=1).dropna().reset_index(drop=True).copy()
urls['campaign_id'] = urls['campaign_id'].astype(int)
urls['id'] = urls['id'].astype(int)
urls['urls'] = urls['urls'].astype(str)
urls = urls.drop_duplicates()

In [23]:
def url_sep2(urls):
    try:
        pattern_full_url = '{\'primary\'.*\'url\': \'(.*?)\',.*'
        pettern_source = '.*utm_source=(.*?)&.*'
        pattern_medium = '.*utm_medium=(.*?)&.*'
        pattern_campaign = '.*utm_campaign=(.*?)&.*'
        pattern_content = '.*utm_content=(.*?)&.*'
        pattern_term = '.*utm_term=(.*?)$'
        urls['urls'] = urls['urls'].astype(str)
        urls['urls'] = urls['urls'].apply(lambda x: find_pattern(x,pattern_full_url)).astype(str)
        urls['utm_s'] = urls['urls'].apply(lambda x: find_pattern(x,pettern_source)).astype(str)
        urls['utm_m'] = urls['urls'].apply(lambda x: find_pattern(x,pattern_medium)).astype(str)
        urls['utm_cam'] = urls['urls'].apply(lambda x: find_pattern(x,pattern_campaign)).astype(str)
        urls['utm_con'] = urls['urls'].apply(lambda x: find_pattern(x,pattern_content)).astype(str)
        urls['utm_t'] = urls['urls'].apply(lambda x: find_pattern(x,pattern_term)).astype(str)
        return urls
    except (KeyError, AttributeError) as e:
        print('end')

clear = url_sep2(urls)

In [24]:
clear = clear.drop_duplicates()
clear

,campaign_id,id,urls,utm_s,utm_m,utm_cam,utm_con,utm_t
0,55628205,111221394,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
1,55628205,111221397,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
2,55628398,111221624,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
3,55628398,111221625,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
4,55628398,111221626,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
...,...,...,...,...,...,...,...,...
187,58003014,114447227,https://app.appsflyer.com/ru.m2.squaremeter?pi...,None,None,None,None,None
189,58003014,114447228,https://app.appsflyer.com/ru.m2.squaremeter?pi...,None,None,None,None,None
191,63580886,121595516,https://app.appsflyer.com/ru.m2.squaremeter?pi...,None,None,None,None,None
193,63580886,121595517,https://app.appsflyer.com/ru.m2.squaremeter?pi...,None,None,None,None,None


In [25]:
start = names.merge(clear, how = 'inner', on = 'campaign_id')

In [26]:
start['urls'] = start['urls'].astype(str)
start['utm_source'] = start['utm_source'].astype(str)

In [27]:
# Замена значения одних колонок на другие. Важна последовательность.
# s.loc[s['n_reviews'] > 120, ['success']] = s['price']

idx = (start.query("utm_source == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_source"] = start.loc[idx, "utm_s"]
idx = (start.query("utm_medium == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_medium"] = start.loc[idx, "utm_m"]
idx = (start.query("utm_campaign == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_campaign"] = start.loc[idx, "utm_cam"]
idx = (start.query("utm_content == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_content"] = start.loc[idx, "utm_con"]
idx = (start.query("utm_term == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_term"] = start.loc[idx, "utm_t"]

idx = (start.query("utm_content == '{{campaign_id}}'").index.to_series().to_numpy())
start.loc[idx, "utm_content"] = start.loc[idx, "campaign_id"]
idx = (start.query("utm_term == '{{banner_id}}'").index.to_series().to_numpy())
start.loc[idx, "utm_term"] = start.loc[idx, "id"]

idx = (start.query("utm_content == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_content"] = start.loc[idx, "campaign_id"]
idx = (start.query("utm_term == 'None'").index.to_series().to_numpy())
start.loc[idx, "utm_term"] = start.loc[idx, "id"]

In [28]:
start.head(5)

,campaign_id,name,utm,utm_source,utm_medium,utm_campaign,utm_content,utm_term,id,urls,utm_s,utm_m,utm_cam,utm_con,utm_t
0,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221394,111221394,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
1,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221397,111221397,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
2,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221624,111221624,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
3,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221625,111221625,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
4,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221626,111221626,http://m2.ru/novostroyki/quiz/,None,None,None,None,None


In [29]:
start['utm_campaign'] = start.utm_campaign.apply(lambda x: 'no campaign name' if x == 'None' else x)
start['utm_source'] = start.utm_source.apply(lambda x: 'target.my.com' if x != 'mytarget' else x)
start['utm_medium'] = start.utm_medium.apply(lambda x: 'cpc' if x == 'cpc' else ('cpm' if x == 'cpm'  else ('cpi' if x == 'cpi'  else 'referral')))

start = start.drop_duplicates()
start.head(15)

,campaign_id,name,utm,utm_source,utm_medium,utm_campaign,utm_content,utm_term,id,urls,utm_s,utm_m,utm_cam,utm_con,utm_t
0,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221394,111221394,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
1,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221397,111221397,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
2,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221624,111221624,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
3,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221625,111221625,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
4,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221626,111221626,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
5,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221627,111221627,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
6,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221628,111221628,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
7,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221629,111221629,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
8,55628659,Перезапуск МО Квиз КвВидео РабочиеВидео2 Ключи,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628659,111221946,111221946,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
9,55628659,Перезапуск МО Квиз КвВидео РабочиеВидео2 Ключи,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628659,111221947,111221947,http://m2.ru/novostroyki/quiz/,None,None,None,None,None


In [30]:
def types(df):
    if '9644e23da9@agency_client' in df:
        return 'Классифайд'
    elif '8ee847846c@agency_client' in df:
        return 'Новостройки'
    elif 'ef427e7659@agency_client' in df:
        return 'Mobile'
    elif 'fa64982560@agency_client' in df:
        return 'Журнал'
    elif '8b193322ff@agency_client' in df:
        return 'B2B'
    else:
        return 'no type'
    raise Exception("какая-то хрень")

money['type'] = money['adAccount'].apply(types)

In [31]:
money

,date,id,base shows,base clicks,base goals,base spent,base cpm,base cpc,base cpa,base ctr,base cr,adAccount,type
0,2022-10-31,111221394,654,1,0,14.32,21.9,14.32,0,0.152905,0.000000,8ee847846c@agency_client,Новостройки
1,2022-10-31,111221397,57,1,1,15.00,263.16,15,15,1.754386,100.000000,8ee847846c@agency_client,Новостройки
2,2022-10-31,111221624,299,11,5,108.94,364.35,9.9,21.79,3.678930,45.454545,8ee847846c@agency_client,Новостройки
3,2022-10-31,111221625,104,2,0,20.00,192.31,10,0,1.923077,0.000000,8ee847846c@agency_client,Новостройки
4,2022-10-31,111221626,175,4,1,40.00,228.57,10,40,2.285714,25.000000,8ee847846c@agency_client,Новостройки
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,2022-11-01,121595516,293,1,1,15.92,54.35,15.92,15.92,0.341297,100.000000,ef427e7659@agency_client,Mobile
193,2022-10-31,121595517,6636,16,0,862.79,130.02,53.92,0,0.241109,0.000000,ef427e7659@agency_client,Mobile
194,2022-11-01,121595517,606,0,0,35.81,59.09,0,0,0.000000,0.000000,ef427e7659@agency_client,Mobile
195,2022-10-31,121595518,2251,9,1,198.64,88.25,22.07,198.64,0.399822,11.111111,ef427e7659@agency_client,Mobile


In [32]:
finish = money.merge(start, how='inner', on = 'id')
finish['date'] = pd.to_datetime(finish['date'], format='%Y-%m-%d')
finish.head(3)

,date,id,base shows,base clicks,base goals,base spent,base cpm,base cpc,base cpa,base ctr,base cr,adAccount,type,campaign_id,name,utm,utm_source,utm_medium,utm_campaign,utm_content,utm_term,urls,utm_s,utm_m,utm_cam,utm_con,utm_t
0,2022-10-31,111221394,654,1,0,14.32,21.9,14.32,0,0.152905,0.000000,8ee847846c@agency_client,Новостройки,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221394,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
1,2022-10-31,111221397,57,1,1,15.00,263.16,15,15,1.754386,100.000000,8ee847846c@agency_client,Новостройки,55628205,Перезапуск МО Квиз Баннер/Тизер Недвижимость,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628205,111221397,http://m2.ru/novostroyki/quiz/,None,None,None,None,None
2,2022-10-31,111221624,299,11,5,108.94,364.35,9.9,21.79,3.678930,45.454545,8ee847846c@agency_client,Новостройки,55628398,Перезапуск МО Квиз ПолВидео Обустройство и ремонт,utm_source=mytarget&utm_medium=cpc&utm_campaig...,mytarget,cpc,m2_social_mytarget_cpc_new-building_quizinhous...,55628398,111221624,http://m2.ru/novostroyki/quiz/,None,None,None,None,None


In [33]:
finish = finish.rename(columns = {
 'id':'banner_id',
 'urls':'url',
 'name':'CampaignName',
 'base shows':'Impressions',
 'base clicks': 'Clicks',
 'base spent': 'Cost'
}).fillna(0)
for_load = finish[['type','date','CampaignName','campaign_id', 'banner_id','url','utm_source','utm_medium', 'utm_campaign', 'utm_content', 'utm_term', 'Impressions', 'Clicks', 'Cost', 'adAccount']].copy()

In [34]:
for_load['utm_content'] = for_load['utm_content'].astype(str)
for_load['utm_term'] = for_load['utm_term'].astype(str)

In [36]:
client.execute('''CREATE TABLE external.MT_ADS(type String, date Date, CampaignName String, campaign_id Int64, 
banner_id  Int64, url String, utm_source String, utm_medium String, utm_campaign String, utm_content String, utm_term String, Impressions Int64, Clicks Int64, Cost Float64, adAccount String) ENGINE = MergeTree() ORDER BY (date, type)'''
)

[]

In [37]:
client.insert_dataframe("INSERT INTO external.MT_ADS VALUES", for_load) #VALUES - DO NOT FORGET TO SET

197

In [38]:
# for_load.to_excel("uoru.xlsx")